In [25]:
import pandas as pd

In [26]:
# 加载情感信号数据
df_sentiment = pd.read_csv('sentiment.csv', parse_dates=['date'])

# 查看前几行数据，确认数据加载正确
print(df_sentiment.head())


        date  sentiment                                     sentiment_dict
0 1996-10-30   0.028881  {'positive': 0.1624157875776291, 'negative': 0...
1 1996-12-04  -0.588398  {'positive': 0.07799529284238815, 'negative': ...
2 1997-01-22   0.123663  {'positive': 0.17666494846343994, 'negative': ...
3 1997-03-12  -0.728463  {'positive': 0.02856595441699028, 'negative': ...
4 1997-05-07  -0.035630  {'positive': 0.16252721846103668, 'negative': ...


In [27]:
# 加载美联储利率决策数据
df_rates = pd.read_csv('historical.csv', parse_dates=['date'])

# 查看前几行数据，确认数据加载正确
print(df_rates.head())


        date  rate_change   decision
0 2024-09-18         -0.5   Decrease
1 2024-07-31          0.0  No Change
2 2024-06-12          0.0  No Change
3 2024-05-01          0.0  No Change
4 2024-03-21          0.0  No Change


In [30]:
# 重命名日期列
df_sentiment.rename(columns={'date': 'sentiment_date'}, inplace=True)
df_rates.rename(columns={'date': 'decision_date'}, inplace=True)

print(df_sentiment.columns)
print(df_rates.columns)



Index(['sentiment_date', 'sentiment', 'sentiment_dict'], dtype='object')
Index(['decision_date', 'rate_change', 'decision'], dtype='object')


In [31]:
# 按日期排序
df_sentiment.sort_values('sentiment_date', inplace=True)
df_rates.sort_values('decision_date', inplace=True)



In [32]:
# 使用merge_asof合并数据
# left=df_rates：左侧是利率决策数据。
# right=df_sentiment：右侧是情感信号数据。
# left_on='date'和right_on='date'：合并的键是日期。
# direction='backward'：对于左侧的每个日期，在右侧寻找小于或等于该日期的最近日期。

df_merged = pd.merge_asof(
    left=df_rates,
    right=df_sentiment,
    left_on='decision_date',
    right_on='sentiment_date',
    direction='backward'
)

print(df_merged.head())



  decision_date  rate_change   decision sentiment_date  sentiment  \
0    1996-11-13         0.00  No Change     1996-10-30   0.028881   
1    1996-12-17         0.00  No Change     1996-12-04  -0.588398   
2    1997-02-05         0.00  No Change     1997-01-22   0.123663   
3    1997-03-25         0.25   Increase     1997-03-12  -0.728463   
4    1997-05-20         0.00  No Change     1997-05-07  -0.035630   

                                      sentiment_dict  
0  {'positive': 0.1624157875776291, 'negative': 0...  
1  {'positive': 0.07799529284238815, 'negative': ...  
2  {'positive': 0.17666494846343994, 'negative': ...  
3  {'positive': 0.02856595441699028, 'negative': ...  
4  {'positive': 0.16252721846103668, 'negative': ...  


In [34]:
# 确保日期列为 datetime 类型
df_merged['decision_date'] = pd.to_datetime(df_merged['decision_date'])
df_merged['sentiment_date'] = pd.to_datetime(df_merged['sentiment_date'])

# 计算日期差
df_merged['days_difference'] = (df_merged['decision_date'] - df_merged['sentiment_date']).dt.days

print(df_merged[['decision_date', 'sentiment_date', 'days_difference']].head())


  decision_date sentiment_date  days_difference
0    1996-11-13     1996-10-30               14
1    1996-12-17     1996-12-04               13
2    1997-02-05     1997-01-22               14
3    1997-03-25     1997-03-12               13
4    1997-05-20     1997-05-07               13


In [35]:
# 过滤掉日期差为负数的行（如果有）
df_merged = df_merged[df_merged['days_difference'] >= 0]


In [36]:
# 保存合并后的数据到 merged.csv
df_merged.to_csv('merged.csv', index=False)
